In [5]:
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import yahoo_fin.stock_info as si
from utils.utils_bigquery import *
from datetime import *

In [6]:
key_path = key_path
project = project_id
dataset = 'bronze'
table = 'bronze_currency_data'
table_conca = f'{project}.{dataset}.{table}'

#schema = [
#    {'name': 'Date', 'type': 'DATE'},
#    {'name': 'currency', 'type': 'STRING'},
#    {'name': 'Adj Close', 'type': 'INTEGER'},
#    {'name': 'Close', 'type': 'INTEGER'},
#    {'name': 'High', 'type': 'INTEGER'},
#    {'name': 'Low', 'type': 'INTEGER'},
#    {'name': 'Open', 'type': 'INTEGER'},
#    {'name': 'Volume', 'type': 'INTEGER'},
#]

In [7]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [8]:
import yfinance as yf
import pandas as pd
from datetime import date


# Definir los pares de divisas para las 10 monedas más importantes
tickers = {
    'Euro (EURUSD)': 'EURUSD=X',
    'Yen Japonés (JPYUSD)': 'JPYUSD=X',
    'Libra Esterlina (GBPUSD)': 'GBPUSD=X',
    'Franco Suizo (USDCHF)': 'USDCHF=X',
    'Dólar Canadiense (USDCAD)': 'USDCAD=X',
    'Dólar Australiano (AUDUSD)': 'AUDUSD=X',
    'Dólar Neozelandés (NZDUSD)': 'NZDUSD=X',
    'Dólar de Hong Kong (USDHKD)': 'USDHKD=X',
    'Dólar de Singapur (USDSGD)': 'USDSGD=X',
    'Rupia India (USDINR)': 'USDINR=X'
}

# Lista para almacenar los DataFrames individuales
df_list = []

# Obtener datos históricos para cada ticker
for name, ticker in tickers.items():
    data = yf.download(ticker, start='2015-01-01', end=date.today())
    data['Currency'] = name  # Añadir columna con el nombre de la moneda
    data.reset_index(inplace=True)  # Convertir el índice 'date' a una columna
    df_list.append(data)

# Concatenar todos los DataFrames en uno solo
df = pd.concat(df_list)

# Imprimir los primeros datos del DataFrame combinado
print("Datos históricos combinados:")
df.head()


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Datos históricos combinados:


,Date,Open,High,Low,Close,Adj Close,Volume,Currency
0,2015-01-01,1.209863,1.209863,1.209863,1.209863,1.209863,0,Euro (EURUSD)
1,2015-01-02,1.208868,1.208956,1.201080,1.208941,1.208941,0,Euro (EURUSD)
2,2015-01-05,1.195500,1.197590,1.188909,1.194643,1.194643,0,Euro (EURUSD)
3,2015-01-06,1.193830,1.197000,1.188693,1.193902,1.193902,0,Euro (EURUSD)
4,2015-01-07,1.187479,1.190000,1.180401,1.187536,1.187536,0,Euro (EURUSD)


In [9]:
df['Date'] = pd.to_datetime(df['Date'])

In [10]:
print(df.dtypes)

Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
Currency             object
dtype: object


In [11]:
df = pd.DataFrame(df)
df

,Date,Open,High,Low,Close,Adj Close,Volume,Currency
0,2015-01-01,1.209863,1.209863,1.209863,1.209863,1.209863,0,Euro (EURUSD)
1,2015-01-02,1.208868,1.208956,1.201080,1.208941,1.208941,0,Euro (EURUSD)
2,2015-01-05,1.195500,1.197590,1.188909,1.194643,1.194643,0,Euro (EURUSD)
3,2015-01-06,1.193830,1.197000,1.188693,1.193902,1.193902,0,Euro (EURUSD)
4,2015-01-07,1.187479,1.190000,1.180401,1.187536,1.187536,0,Euro (EURUSD)
...,...,...,...,...,...,...,...,...
2502,2024-08-09,83.988998,83.993103,83.879997,83.988998,83.988998,0,Rupia India (USDINR)
2503,2024-08-12,83.949799,83.964699,83.821503,83.949799,83.949799,0,Rupia India (USDINR)
2504,2024-08-13,83.889999,83.970001,83.887001,83.889999,83.889999,0,Rupia India (USDINR)
2505,2024-08-14,83.882004,83.975502,83.889000,83.882004,83.882004,0,Rupia India (USDINR)


In [14]:
# Guardamos los datos en bigquery
bigquery.save_dataframe(df, project, dataset, table, if_exists='replace', schema=None)

100%|██████████| 1/1 [00:00<?, ?it/s]
